# Выбор числа соседей

- *В этом задании вам нужно подобрать оптимальное значение k для алгоритма kNN. Будем использовать набор данных Wine, где требуется предсказать сорт винограда, из которого изготовлено вино, используя результаты химических анализов.*
1. Загрузите выборку Wine по адресу https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data
2. Извлеките из данных признаки и классы. Класс записан в первом столбце (три варианта), признаки — в столбцах со второго по последний. Более подробно о сути признаков можно прочитать по адресу https://archive.ics.uci.edu/ml/datasets/Wine (см. также файл wine.names, приложенный к заданию)
3. Оценку качества необходимо провести методом кроссвалидации по 5 блокам (5-fold). Создайте генератор разбиений, который перемешивает выборку перед формированием блоков (shuffle=True). Для воспроизводимости результата, создавайте генератор KFold с фиксированным параметром random_state=42. В качестве меры качества используйте долю верных ответов (accuracy).
4. Найдите точность классификации на кросс-валидации для метода k ближайших соседей (sklearn.neighbors.KNeighborsClassifier), при k от 1 до 50. При каком k получилось оптимальное качество? Чему оно равно (число в интервале от 0 до 1)? Данные результаты и будут ответами на вопросы 1 и 2.
5. Произведите масштабирование признаков с помощью функции sklearn.preprocessing.scale. Снова найдите оптимальное k на кроссвалидации.
6. Какое значение k получилось оптимальным после приведения признаков к одному масштабу? Как изменилось значение качества? Приведите ответы на вопросы 3 и 4.

In [41]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score  # вычислить качество на всех k разбиениях 

# нормирование признаков:
# на входе: матрицу признаков
# на выходе: масштабированная матрица, в которой каждый столбец имеет нулевое среднее значение и единичное стандартное отклонение
from sklearn.preprocessing import scale              

In [85]:
data = np.genfromtxt('data/wine.data', delimiter=',')
y = np.array(data[:,0], dtype=int)
X = np.array(data[:, 1:])
kf = KFold(n_splits=5, shuffle=True, random_state=42)

print('Без нормирования признаков:')
k_neigh = 0
max_score = 0
for k in range(1, 51):
    k_neighbors_model = KNeighborsClassifier(n_neighbors=k);
    scores = cross_val_score(k_neighbors_model, X, y, cv=kf, scoring='accuracy')
    if np.mean(scores) > max_score:
        max_score = np.mean(scores)
        k_neigh = k

print(k_neigh, 'neighbors with accuracy', max_score)
print()

print('После нормирования признаков:')

X = scale(X, axis=0, with_mean=True, with_std=True)

k_neigh = 0
max_score = 0
for k in range(1, 51):
    k_neighbors_model = KNeighborsClassifier(n_neighbors=k);
    scores = cross_val_score(k_neighbors_model, X, y, cv=kf, scoring='accuracy')
    if np.mean(scores) > max_score:
        max_score = np.mean(scores)
        k_neigh = k
        
print(k_neigh, 'neighbors with accuracy', max_score)

Без нормирования признаков:
1 neighbors with accuracy 0.7304761904761905

После нормирования признаков:
29 neighbors with accuracy 0.9776190476190475


# Выбор Метрики

1. Загрузите выборку Boston с помощью функции sklearn.datasets.load_boston(). Результатом вызова данной функции является объект, у которого признаки записаны в поле data, а целевой вектор — в поле target.
2. Приведите признаки в выборке к одному масштабу при помощи функции sklearn.preprocessing.scale.
3. Переберите разные варианты параметра метрики p по сетке от 1 до 10 с таким шагом, чтобы всего было протестировано 200 вариантов (используйте функцию numpy.linspace). Используйте KNeighborsRegressor с n_neighbors=5 и weights='distance' — данный параметр добавляет в алгоритм веса, зависящие от расстояния до ближайших соседей. В качестве метрики качества используйте среднеквадратичную ошибку (параметр scoring='mean_squared_error' у cross_val_score; при использовании библиотеки scikit-learn версии 0.18.1 и выше необходимо указывать scoring='neg_mean_squared_error').  Качество оценивайте, как и в предыдущем задании, с помощью кросс-валидации по 5 блокам с random_state = 42, не забудьте включить перемешивание выборки (shuffle=True).
4. Определите, при каком p качество на кросс-валидации оказалось оптимальным. Обратите внимание, что cross_val_score возвращает массив показателей качества по блокам; необходимо максимизировать среднее этих показателей. Это значение параметра и будет ответом на задачу.

In [51]:
from sklearn import datasets
from sklearn.neighbors import KNeighborsRegressor

In [86]:
data = datasets.load_boston()
X = data['data']
y = data['target']
X = scale(X, with_mean=True, with_std=True)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

param = 0
max_score = None
for p_value in np.linspace(1, 10, 200):
    k_neighbors_model = KNeighborsRegressor(n_neighbors=5, weights='distance', metric='minkowski', p=p_value) # weights='' добавляет в алгоритм веса, зависящие от расстояния до ближайших соседей
    scores = cross_val_score(k_neighbors_model, X, y, cv=kf, scoring='neg_mean_squared_error')
    if max_score == None or np.max(scores) > max_score:
        max_score = np.max(scores)
        param = p_value

    
print(param, 'param with max accuracy', max_score)

1.0 param with max accuracy -11.833733363248196
